# Análisis de Datos SQL en Jupyter Notebook
Este notebook contiene el análisis de datos utilizando **SQLite** y **Python**.

## Objetivos de la Prueba Técnica
1. Crear una base de datos SQL usando los archivos CSV proporcionados.
2. Crear las tablas necesarias y cargar los datos.
3. Identificar los jugadores que hicieron su primer depósito (FTD).
4. Filtrar aquellos que cumplen la condición de CPA (total de depósitos mayor a 100 USD).
5. Generar reportes de la cantidad de FTD por mes y de CPA por mes.

---

In [2]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 4.2 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
import sqlite3
import pandas as pd

## 1. Cargar los Datos desde los Archivos CSV
En esta sección, cargamos los datos desde los archivos CSV en pandas.

In [4]:
players_df = pd.read_csv('./data/players.csv')
traffic_sources_df = pd.read_csv('./data/traffic_sources.csv')
deposits_df = pd.read_csv('./data/deposits.csv')
internal_records_df = pd.read_csv('./data/internal_records.csv')

# Mostrar las primeras filas de cada dataset
players_df.head(), traffic_sources_df.head(), deposits_df.head(), internal_records_df.head()

(                              player_id              name registration_date  \
 0  23b8c1e9-3924-46de-beb1-3b9046685257    Derek Anderson        2023-07-27   
 1  e465e150-bd9c-46b3-ad3c-2d6d1a3d1fa7  Michael Martinez        2023-02-28   
 2  0822e8f3-6c03-4199-972a-846916419f82     Cynthia Moore        2022-01-24   
 3  815ef6d1-3b8f-4a18-b7f8-a88b17fc695a      Jason Walker        2023-04-13   
 4  b74d0fb1-32e7-4629-8fad-c1a606cb0fb3      Julia Wilson        2023-05-20   
 
                 country  
 0                France  
 1                  Niue  
 2                Mexico  
 3            Tajikistan  
 4  Syrian Arab Republic  ,
                               player_id    trafficker  cost_of_acquisition  \
 0  23b8c1e9-3924-46de-beb1-3b9046685257  Trafficker_5               111.22   
 1  e465e150-bd9c-46b3-ad3c-2d6d1a3d1fa7  Trafficker_3               234.12   
 2  0822e8f3-6c03-4199-972a-846916419f82  Trafficker_9                71.73   
 3  815ef6d1-3b8f-4a18-b7f8-a88b17fc695

## 2. Crear Base de Datos y Tablas SQL
Creamos una base de datos en SQLite y definimos las tablas necesarias.

In [5]:
db_conn = sqlite3.connect("gaming_data.db")
cursor = db_conn.cursor()

cursor.executescript('''
    CREATE TABLE IF NOT EXISTS players (
        player_id INTEGER PRIMARY KEY,
        registration_date DATE
    );
    CREATE TABLE IF NOT EXISTS traffic_sources (
        player_id INTEGER,
        source TEXT,
        FOREIGN KEY (player_id) REFERENCES players(player_id)
    );
    CREATE TABLE IF NOT EXISTS deposits (
        deposit_id INTEGER PRIMARY KEY,
        player_id INTEGER,
        deposit_date DATE,
        amount REAL,
        FOREIGN KEY (player_id) REFERENCES players(player_id)
    );
    CREATE TABLE IF NOT EXISTS internal_records (
        player_id INTEGER PRIMARY KEY,
        status TEXT
    );
''')
db_conn.commit()
print("Base de datos y tablas creadas correctamente.")

Base de datos y tablas creadas correctamente.


## 3. Cargar Datos en la Base de Datos
Insertamos los datos de los DataFrames en las tablas SQLite.

In [ ]:
players_df.to_sql("players", db_conn, if_exists="replace", index=False)
traffic_sources_df.to_sql("traffic_sources", db_conn, if_exists="replace", index=False)
deposits_df.to_sql("deposits", db_conn, if_exists="replace", index=False)
internal_records_df.to_sql("internal_records", db_conn, if_exists="replace", index=False)
print("Datos cargados exitosamente en SQLite.")

## 4. Identificar Primer Depósito (FTD)
Identificamos el primer depósito de cada jugador utilizando `MIN(deposit_date)`.

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS first_time_deposits AS
    SELECT player_id, MIN(deposit_date) AS first_deposit_date
    FROM deposits
    GROUP BY player_id;
''')
db_conn.commit()
print("Tabla de FTD creada correctamente.")

## 5. Filtrar Jugadores CPA (Depósitos > 100 USD)
Seleccionamos los jugadores que han depositado más de 100 USD en total.

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS cpa_players AS
    SELECT player_id, SUM(amount) AS total_deposits
    FROM deposits
    GROUP BY player_id
    HAVING total_deposits > 100;
''')
db_conn.commit()
print("Tabla de CPA creada correctamente.")

## 6. Reporte de FTD por Mes
Generamos un reporte de la cantidad de jugadores que hicieron su primer depósito cada mes.

In [ ]:
ftd_report = pd.read_sql('''
    SELECT strftime('%Y-%m', first_deposit_date) AS month, COUNT(player_id) AS ftd_count
    FROM first_time_deposits
    GROUP BY month
    ORDER BY month;
''', db_conn)
print(ftd_report)

## 7. Reporte de CPA por Mes
Generamos un reporte de la cantidad de jugadores CPA (depósitos > 100 USD) cada mes.

In [ ]:
cpa_report = pd.read_sql('''
    SELECT strftime('%Y-%m', deposit_date) AS month, COUNT(DISTINCT deposits.player_id) AS cpa_count
    FROM deposits
    JOIN cpa_players ON deposits.player_id = cpa_players.player_id
    GROUP BY month
    ORDER BY month;
''', db_conn)
print(cpa_report)